# Spar Nord Bank - ETL Project

Spar Nord Bank aims to analyze withdrawal patterns and identify key factors influencing them to enhance the efficiency of ATM refill management. Additionally, the bank seeks to derive valuable insights from the data. Spar Nord Bank has developed a dimensional model datastore, known as the ATM Data Mart, specifically designed for analyzing ATM transaction data and understanding usage patterns. The project will leverage this predefined schema (target schema) within the ATM Data Mart for its implementation.

In [2]:
#Starting spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('jupyter_Spark').master("local").getOrCreate()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Reading the input data from hdfs
df = spark.read.csv("/user/root/spar_nord_bank_atm/part-m-00000", header = False, inferSchema = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.show(1)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+------+---+------+---+---+----------+-----------+----+----+------+------+----+----------+----+----------+----+----+-----+------+-------+--------+------+----+----+----+----+-----+----+----+----+----------+
| _c0|    _c1|_c2|   _c3|_c4|   _c5|_c6|_c7|       _c8|        _c9|_c10|_c11|  _c12|  _c13|_c14|      _c15|_c16|      _c17|_c18|_c19| _c20|  _c21|   _c22|    _c23|  _c24|_c25|_c26|_c27|_c28| _c29|_c30|_c31|_c32|      _c33|
+----+-------+---+------+---+------+---+---+----------+-----------+----+----+------+------+----+----------+----+----------+----+----+-----+------+-------+--------+------+----+----+----+----+-----+----+----+----+----------+
|2017|January|  1|Sunday|  0|Active|  1|NCR|NÃƒÂ¦stved|Farimagsvej|   8|4700|55.233|11.763| DKK|MasterCard|5643|Withdrawal|null|null|55.23|11.761|2616038|Naestved|281.15|1014|  87|   7| 260|0.215|  92| 500|Rain|light rain|
+----+-------+---+------+---+------+---+---+----------+-----------+----+----+------+------+----+----------+-

### Creating  input schema using StructType and read the data

In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
Schema = StructType([StructField('year', IntegerType(), nullable = True),
                        StructField('month', StringType(), True),
                        StructField('day', IntegerType(), True),
                        StructField('weekday', StringType(), True),
                        StructField('hour', IntegerType(), True),
                        StructField('atm_status', StringType(), True),
                        StructField('atm_id', StringType(), True),
                        StructField('atm_manufacturer', StringType(), True),
                        StructField('atm_location', StringType(), True),
                        StructField('atm_streetname', StringType(), True),
                        StructField('atm_street_number', IntegerType(), True),
                        StructField('atm_zipcode', IntegerType(), True),
                        StructField('atm_lat', DoubleType(), True),
                        StructField('atm_lon', DoubleType(), True),
                        StructField('currency', StringType(), True),
                        StructField('card_type', StringType(), True),
                        StructField('transaction_amount', IntegerType(), True),
                        StructField('service', StringType(), True),
                        StructField('message_code', StringType(), True),
                        StructField('message_text', StringType(), True),
                        StructField('weather_lat', DoubleType(), True),
                        StructField('weather_lon', DoubleType(), True),
                        StructField('weather_city_id', IntegerType(), True),
                        StructField('weather_city_name', StringType(), True),
                        StructField('temp', DoubleType(), True),
                        StructField('pressure', IntegerType(), True),
                        StructField('humidity', IntegerType(), True),
                        StructField('wind_speed', IntegerType(), True),
                        StructField('wind_deg', IntegerType(), True),
                        StructField('rain_3h', DoubleType(), True),
                        StructField('clouds_all', IntegerType(), True),
                        StructField('weather_id', IntegerType(), True),
                        StructField('weather_main', StringType(), True),
                        StructField('weather_description', StringType(), True)])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Loading data to Schema and verifying the count

In [7]:
df = spark.read.csv("/user/root/spar_nord_bank_atm/part-m-00000", header = False, schema = Schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Validation 1 - count of rows

In [8]:
df.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [9]:
#Displaying the Schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- hour: integer (nullable = true)
 |-- atm_status: string (nullable = true)
 |-- atm_id: string (nullable = true)
 |-- atm_manufacturer: string (nullable = true)
 |-- atm_location: string (nullable = true)
 |-- atm_streetname: string (nullable = true)
 |-- atm_street_number: integer (nullable = true)
 |-- atm_zipcode: integer (nullable = true)
 |-- atm_lat: double (nullable = true)
 |-- atm_lon: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- card_type: string (nullable = true)
 |-- transaction_amount: integer (nullable = true)
 |-- service: string (nullable = true)
 |-- message_code: string (nullable = true)
 |-- message_text: string (nullable = true)
 |-- weather_lat: double (nullable = true)
 |-- weather_lon: double (nullable = true)
 |-- weather_city_id: integer (nullable = true)
 |-- weather_city_name: st

### Creating the Dimension and Fact tables

#### The four dimension tables to be created -
##### DIM_LOCATION
##### DIM_ATM
##### DIM_DATE
##### DIM_CARD_TYPE

#### Creating dimension table - DIM_LOCATION

In [11]:
# creating a temporary df and selecting required columns & making sure records are distinct
location = df.select('atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'atm_lat', 'atm_lon').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# creating the primary key column
df_temp = location.rdd.zipWithIndex().toDF()
dim_location = df_temp.select(col("_1.*"),col("_2").alias('location_id'))
dim_location.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
|    atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|location_id|
+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
|         Kolding|           Vejlevej|              135|       6000| 55.505|  9.457|          0|
|  Skelagervej 15|        Skelagervej|               15|       9000| 57.023|  9.891|          1|
|Intern HolbÃƒÂ¦k|        Slotsvolden|                7|       4300| 55.718| 11.704|          2|
|          Odense|       FÃƒÂ¦lledvej|                3|       5000| 55.394|  10.37|          3|
|           Ikast|RÃƒÂ¥dhusstrÃƒÂ¦det|               12|       7430| 56.139|  9.154|          4|
+----------------+-------------------+-----------------+-----------+-------+-------+-----------+
only showing top 5 rows

In [14]:
# renaming the columns as per requirement
DIM_LOCATION = dim_location.withColumnRenamed('atm_location','location').withColumnRenamed('atm_streetname','streetname').withColumnRenamed('atm_street_number','street_number').withColumnRenamed('atm_zipcode','zipcode').withColumnRenamed('atm_lat','lat').withColumnRenamed('atm_lon','lon')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# rearranging the columns according to the target model
DIM_LOCATION = DIM_LOCATION.select('location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# checking that all required columns are present and named correctly
DIM_LOCATION.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon']

#### Validation 2 - count for the Location  Dimension

In [17]:
# validating the count of the dataframe
DIM_LOCATION.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

### Creating dimension table - DIM_ATM

In [18]:
# creating a temporary df and selecting required columns
atm = df.select('atm_id', 'atm_manufacturer', 'atm_lat', 'atm_lon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# renaming the column atm_id to atm_number as per requirement
atm = atm.withColumnRenamed('atm_id', 'atm_number')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# join the dim_location and atm dataframes
atm = atm.join(dim_location, on = ['atm_lat', 'atm_lon'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# validating the columns after joining
atm.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_lat', 'atm_lon', 'atm_number', 'atm_manufacturer', 'atm_location', 'atm_streetname', 'atm_street_number', 'atm_zipcode', 'location_id']

In [22]:
# selecting the required columns and making sure records are distinct
atm = atm.select('atm_number', 'atm_manufacturer', 'location_id').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# renaming the columns as per requirement
atm = atm.withColumnRenamed('location_id', 'atm_location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# creating the primary key column
df_temp = atm.rdd.zipWithIndex().toDF()
dim_atm = df_temp.select(col("_1.*"),col("_2").alias('atm_id'))
dim_atm.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+---------------+------+
|atm_number|atm_manufacturer|atm_location_id|atm_id|
+----------+----------------+---------------+------+
|       100|             NCR|             70|     0|
|        60|             NCR|             68|     1|
|        45|             NCR|             66|     2|
|        54|             NCR|              9|     3|
|        53|             NCR|             27|     4|
+----------+----------------+---------------+------+
only showing top 5 rows

In [25]:
# rearranging the columns according to the target model
DIM_ATM = dim_atm.select('atm_id', 'atm_number', 'atm_manufacturer', 'atm_location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# validating required columns are present and named correctly
DIM_ATM.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_id', 'atm_number', 'atm_manufacturer', 'atm_location_id']

#### Validation 3 - count for the ATM Dimension

In [27]:
# validating the count of the dataframe
DIM_ATM.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

156

### Creating dimension table - DIM_DATE

In [49]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
# Creating date datafraame by selecting the required columns
date = df.select("year","month","day","hour","weekday").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
# Creating a temp column for concatenating the date
date = date.withColumn("full_date", concat_ws('-', date.year, date.month, date.day))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
# Creating a temp column for concatenating the time
date = date.withColumn('time', concat_ws(':', date.hour, lit('00'), lit('00')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
# Creating column for the table to show full timestame 
date = date.withColumn("full_date_time",concat_ws(" ",date.full_date, date.time))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
# Casting the value for timestamp in datatype timestampe in a specific pattern
pattern = 'yyyy-MMM-dd HH:mm:ss'
date = date.withColumn('full_date_time', unix_timestamp(date.full_date_time, pattern).cast('timestamp'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
# Selecting the distinct records
date = date.select('year', 'month', 'day', 'hour', 'weekday', 'full_date_time')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
# Creating the primary key and showing top 5 records
date_temp = date.rdd.zipWithIndex().toDF()
DIM_DATE = date_temp.select(col("_1.*"),col("_2").alias('date_id'))
DIM_DATE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+--------+-------------------+-------+
|year|  month|day|hour| weekday|     full_date_time|date_id|
+----+-------+---+----+--------+-------------------+-------+
|2017|January|  5|  21|Thursday|2017-01-05 21:00:00|      0|
|2017|January| 22|  15|  Sunday|2017-01-22 15:00:00|      1|
|2017|  April|  7|   9|  Friday|2017-04-07 09:00:00|      2|
|2017|January| 23|  18|  Monday|2017-01-23 18:00:00|      3|
|2017|  March| 17|   1|  Friday|2017-03-17 01:00:00|      4|
+----+-------+---+----+--------+-------------------+-------+
only showing top 5 rows

In [57]:
DIM_DATE = DIM_DATE.select('date_id', 'full_date_time', 'year', 'month', 'day', 'hour', 'weekday').distinct()
DIM_DATE.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['date_id', 'full_date_time', 'year', 'month', 'day', 'hour', 'weekday']

In [58]:
DIM_DATE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+--------+---+----+---------+
|date_id|     full_date_time|year|   month|day|hour|  weekday|
+-------+-------------------+----+--------+---+----+---------+
|    267|2017-03-04 03:00:00|2017|   March|  4|   3| Saturday|
|    693|2017-02-07 10:00:00|2017|February|  7|  10|  Tuesday|
|   4104|2017-06-09 21:00:00|2017|    June|  9|  21|   Friday|
|   4297|2017-05-31 22:00:00|2017|     May| 31|  22|Wednesday|
|   5186|2017-07-22 13:00:00|2017|    July| 22|  13| Saturday|
+-------+-------------------+----+--------+---+----+---------+
only showing top 5 rows

#### Validation 4 - count for the Date Dimension

In [59]:
# Validating the record counts
DIM_DATE.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

### Creating dimension table - DIM_CARD_TYPE

In [60]:
# Creating the dataframe card_type by selecting distinct records
card = df.select('card_type').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
# Creating primary key and showing top 5 rows
card_temp = card.rdd.zipWithIndex().toDF()
DIM_CARD_TYPE = card_temp.select(col("_1.*"),col("_2").alias('card_type_id'))
DIM_CARD_TYPE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+
|           card_type|card_type_id|
+--------------------+------------+
|Visa Dankort - on-us|           0|
|  Mastercard - on-us|           1|
|         HÃƒÂ¦vekort|           2|
|            VisaPlus|           3|
|     Dankort - on-us|           4|
+--------------------+------------+
only showing top 5 rows

In [62]:
# Rearranging columns and showing column names 
DIM_CARD_TYPE = DIM_CARD_TYPE.select('card_type_id', 'card_type')
DIM_CARD_TYPE.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type_id', 'card_type']

In [63]:
DIM_CARD_TYPE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------------+
|card_type_id|           card_type|
+------------+--------------------+
|           0|Visa Dankort - on-us|
|           1|  Mastercard - on-us|
|           2|         HÃƒÂ¦vekort|
|           3|            VisaPlus|
|           4|     Dankort - on-us|
+------------+--------------------+
only showing top 5 rows

### Validation 5 - count for the Card Type Dimension

In [64]:
# Validating the total records 
DIM_CARD_TYPE.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

### Creating Transaction Fact table - FACT_ATM_TRANS as per the target model

In [65]:
# Creating new dataframe based on the new column names as present in DIM_LOCATION dataframe
fact_location = df.withColumnRenamed('atm_location','location').withColumnRenamed('atm_streetname','streetname').withColumnRenamed('atm_street_number','street_number').withColumnRenamed('atm_zipcode','zipcode').withColumnRenamed('atm_lat','lat').withColumnRenamed('atm_lon','lon')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
# joining the dataframes 
fact_location = fact_location.join(DIM_LOCATION, on = ['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
# showing all the column names
fact_location.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'atm_id', 'atm_manufacturer', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'location_id']

#### Validation 6.1 - count for the all the Stages in the creation of Transaction Fact table

In [68]:
# Validating the total records
fact_location.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [69]:
# Renaming the column names based on the DIM_ATM column names
fact_location = fact_location.withColumnRenamed('atm_id', 'atm_number').withColumnRenamed('location_id', 'atm_location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
# Creating new dataframe by Joining fact_location and DIM_ATM
fact_atm = fact_location.join(DIM_ATM, on = ['atm_number', 'atm_manufacturer', 'atm_location_id'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
# Renaming the column names
fact_atm = fact_atm.withColumnRenamed('atm_location_id', 'weather_loc_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Validation 6.2 - count for the all the Stages in the creation of Transaction Fact table

In [72]:
# Validating the total records
fact_atm.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [73]:
# joining with DIM_DATE
fact_date = fact_atm.join(DIM_DATE, on = ['year', 'month', 'day', 'hour', 'weekday'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Validation 6.3 - count for the all the Stages in the creation of Transaction Fact table

In [74]:
# Validating the total records
fact_date.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [75]:
# joining with DIM_CARD_TYPE
fact_atm_trans = fact_date.join(DIM_CARD_TYPE, on = ['card_type'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Validation 6.4 - count for the all the Stages in the creation of Transaction Fact table

In [76]:
# Validating the total records
fact_atm_trans.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [77]:
# Showing column names
fact_atm_trans.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time', 'card_type_id']

In [78]:
from pyspark.sql.window import Window

w = Window().orderBy('date_id')
FACT_ATM_TRANS = fact_atm_trans.withColumn("trans_id", row_number().over(w))
FACT_ATM_TRANS.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------+---+----+--------+----------+----------------+--------------+----------+---------------+-------------+-------+------+------+----------+--------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+-------+--------+--------+----------+--------+-------+----------+----------+------------+-------------------+------+-------+-------------------+------------+--------+
|           card_type|year|  month|day|hour| weekday|atm_number|atm_manufacturer|weather_loc_id|  location|     streetname|street_number|zipcode|   lat|   lon|atm_status|currency|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|   temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main|weather_description|atm_id|date_id|     full_date_time|card_type_id|trans_id|
+--------------------+----+-------+---+----+--------+----------+----------------

#### Validation 7 - count for the FACT_ATM_TRANS table

In [79]:
#Validating the total records count
FACT_ATM_TRANS.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [80]:
#printing the column names for transaction table
FACT_ATM_TRANS.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'streetname', 'street_number', 'zipcode', 'lat', 'lon', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time', 'card_type_id', 'trans_id']

In [81]:
# Selecting the required columns
FACT_ATM_TRANS = FACT_ATM_TRANS.select('trans_id', 'atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 
'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 
'clouds_all', 'weather_id', 'weather_main', 'weather_description')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
# showing the required column names list
FACT_ATM_TRANS.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['trans_id', 'atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description']

In [84]:
# Validating the total records
FACT_ATM_TRANS.select('*').count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [85]:
FACT_ATM_TRANS.show(5,True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|trans_id|atm_id|weather_loc_id|date_id|card_type_id|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+--------+------+--------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|       1|   106|            11|      0|           0|    Active|     DKK|Withdrawal|              2559|        null|        null|    0.0|         0|       800|       Clear|       Sky is Clear|
|       2|    20|            98|      0|           8|    Active|     DKK|Withdrawal|              3264|        null|        null|    0.0|         0|       800|       Clear|       Sky is Clear|
|       3|   144|            18|   

## Writing the data in the S3 bucket folders

In [92]:
# writing the pyspark dataframe in the S3 bucket - etlprojectpurvi in the folder dim_location
DIM_LOCATION.coalesce(1).write.format('csv').option('header','false').save('s3://etlprojectatm/dim_location/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
# writing the pyspark dataframe in the S3 bucket - etlprojectpurvi in the folder dim_atm
DIM_ATM.coalesce(1).write.format('csv').option('header','false').save('s3://etlprojectatm/dim_atm/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
# writing the pyspark dataframe in the S3 bucket - etlprojectpurvi in the folder dim_date
DIM_DATE.coalesce(1).write.format('csv').option('header','false').save('s3://etlprojectatm/dim_date/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
# writing the pyspark dataframe in the S3 bucket - etlprojectpurvi in the folder dim_card_type
DIM_CARD_TYPE.coalesce(1).write.format('csv').option('header','false').save('s3://etlprojectatm/dim_card_type/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
# writing the pyspark dataframe in the S3 bucket - etlprojectpurvi in the folder fact_atm_trans
FACT_ATM_TRANS.coalesce(1).write.format('csv').option('header','false').save('s3://etlprojectatm/fact_atm_trans/', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…